## Q2:

**For each subway line and month, how many delay-causing incidents occurred, and how did it impact additional wait time for riders?**

In [27]:
import pandas as pd

delayed_df = pd.read_csv("raw_data/delayed.csv")
delayed_df

,month,division,line,day_type,incidents,reporting_category
0,2024-01-01T00:00:00.000,A DIVISION,1,1,0,NaN
1,2024-01-01T00:00:00.000,A DIVISION,1,1,31,Crew Availability
2,2024-01-01T00:00:00.000,A DIVISION,1,1,2,External Factors
3,2024-01-01T00:00:00.000,A DIVISION,1,1,92,Infrastructure & Equipment
4,2024-01-01T00:00:00.000,A DIVISION,1,1,86,Operating Conditions
...,...,...,...,...,...,...
3761,2024-12-01T00:00:00.000,B DIVISION,S ROCK,2,16,External Factors
3762,2024-12-01T00:00:00.000,B DIVISION,S ROCK,2,6,Infrastructure & Equipment
3763,2024-12-01T00:00:00.000,B DIVISION,S ROCK,2,1,Operating Conditions
3764,2024-12-01T00:00:00.000,B DIVISION,S ROCK,2,1,Planned ROW Work


In [4]:
#redoing the cleaning I did in the MTA_Code notebook because it didnt transfer over

#Clean line column for syntax issues (capitalization and white space from front and back)
delayed_df['line'] = delayed_df['line'].str.strip().str.upper()

In [28]:
metrics_df = pd.read_csv("raw_data/metrics.csv")
metrics_df

,month,division,line,period,num_passengers,additional_platform_time,additional_train_time,total_apt,total_att,over_five_mins,over_five_mins_perc,customer_journey_time
0,2024-01-01T00:00:00.000,A DIVISION,1,offpeak,4873143.500,1.656052,0.796657,8.070179e+06,3882225.200,763100.9000,0.156593,0.843407
1,2024-01-01T00:00:00.000,A DIVISION,1,peak,4765373.000,2.115036,0.873300,1.007894e+07,4161600.200,743524.6000,0.156027,0.843973
2,2024-01-01T00:00:00.000,A DIVISION,2,offpeak,2878163.800,1.606833,0.501318,4.624728e+06,1442874.900,487204.1000,0.169276,0.830724
3,2024-01-01T00:00:00.000,A DIVISION,2,peak,3092725.500,2.073322,0.561746,6.412216e+06,1737327.100,460791.2000,0.148992,0.851008
4,2024-01-01T00:00:00.000,A DIVISION,3,offpeak,2125353.800,1.249190,0.328101,2.654971e+06,697329.800,265963.7000,0.125139,0.874861
...,...,...,...,...,...,...,...,...,...,...,...,...
571,2024-12-01T00:00:00.000,B DIVISION,S Fkln,peak,194189.340,0.570730,0.057167,1.108297e+05,11101.285,9461.5890,0.048724,0.951276
572,2024-12-01T00:00:00.000,B DIVISION,S Rock,offpeak,26513.352,1.262324,-0.908065,3.346844e+04,-24075.834,3445.4740,0.129952,0.870048
573,2024-12-01T00:00:00.000,B DIVISION,S Rock,peak,19257.482,0.878703,-1.183044,1.692160e+04,-22782.447,1812.6135,0.094125,0.905875
574,2024-12-01T00:00:00.000,B DIVISION,W,offpeak,1265461.600,0.938558,0.257632,1.187710e+06,326022.940,142330.0200,0.112473,0.887527


In [16]:
!pip install duckdb

                                              0.0/12.3 MB ? eta -:--:--
                                             0.0/12.3 MB 320.0 kB/s eta 0:00:39
                                             0.1/12.3 MB 871.5 kB/s eta 0:00:15
     --                                       0.8/12.3 MB 6.3 MB/s eta 0:00:02
     ------                                   1.9/12.3 MB 11.3 MB/s eta 0:00:01
     -----------                              3.6/12.3 MB 16.5 MB/s eta 0:00:01
     ---------------                          4.9/12.3 MB 18.2 MB/s eta 0:00:01
     --------------------                     6.4/12.3 MB 21.4 MB/s eta 0:00:01
     -------------------------                8.0/12.3 MB 23.1 MB/s eta 0:00:01
     -------------------------------          9.8/12.3 MB 25.1 MB/s eta 0:00:01
     ------------------------------------    11.7/12.3 MB 36.4 MB/s eta 0:00:01
     --------------------------------------  12.3/12.3 MB 38.5 MB/s eta 0:00:01
     --------------------------------------- 12.3

In [25]:
import duckdb

In [ ]:
result = duckdb.query("""
    SELECT
        d.line,
        d.month,
        d.total_incidents,
        m.total_apt,
        m.total_passengers,
        m.avg_additional_platform_time
    FROM (
        -- incidents by line + month
        SELECT
            line,
            month,
            SUM(incidents) AS total_incidents
        FROM delayed_df
        GROUP BY line, month
    ) AS d
    JOIN (
        -- metrics by line + month
        SELECT
            line,
            month,
            SUM(total_apt) AS total_apt,
            SUM(num_passengers) AS total_passengers,
            SUM(num_passengers * additional_platform_time)
                / SUM(num_passengers) AS avg_additional_platform_time
        FROM metrics_df
        GROUP BY line, month
    ) AS m
      ON d.line = m.line
     AND d.month = m.month
    ORDER BY d.line, d.month;
""").to_df()

result


,line,month,total_incidents,total_apt,total_passengers,avg_additional_platform_time
0,1,2024-01-01T00:00:00.000,401.0,18149114.0,9638516.5,1.882978
1,1,2024-02-01T00:00:00.000,429.0,10495347.0,8230996.5,1.275100
2,1,2024-03-01T00:00:00.000,375.0,8006229.5,7250264.4,1.104267
3,1,2024-04-01T00:00:00.000,351.0,14979411.5,9816352.5,1.525965
4,1,2024-05-01T00:00:00.000,382.0,11646732.5,9323154.0,1.249227
...,...,...,...,...,...,...
235,R,2024-08-01T00:00:00.000,260.0,11419876.5,6722566.8,1.698738
236,R,2024-09-01T00:00:00.000,242.0,11274872.5,6791100.4,1.660242
237,R,2024-10-01T00:00:00.000,281.0,12709424.5,7811962.6,1.626918
238,R,2024-11-01T00:00:00.000,238.0,12534280.0,7262751.0,1.725831


## Data Analysis

## Data Visualization